In [1]:
import pandas as pd
import re
import numpy as np
import yaml  # To read YAML configuration files
from pathlib import Path  # For cross-platform file path handling


In [2]:
# Load directory paths from configuration file
with open('paths.yml', 'r') as file:
    paths = yaml.safe_load(file)  # Read and parse YAML file

# Create Path objects for each directory
raw = Path(paths['data']['raw'])  # Directory with raw data
temp = Path(paths['data']['temp'])  # Directory with temporary processed data
processed = Path(paths['data']['processed'])  # Directory with final processed data

In [3]:

# Diccionario para convertir texto a números
str_num = {
    'un': 1, 'uno': 1, 'una': 1, 'dos': 2, 'tres': 3, 'cuatro': 4, 'cinco': 5,
    'seis': 6, 'siete': 7, 'ocho': 8, 'nueve': 9, 'diez': 10,
    'once': 11, 'doce': 12, 'trece': 13, 'catorce': 14, 'quince': 15,
    'dieciseis': 16, 'dieciséis': 16, 'diecisiete': 17, 'dieciocho': 18, 'diecinueve': 19,
    'veinte': 20, 'veintiuno': 21, 'veintiún': 21, 'veintidos': 22, 'veintidós': 22,
    'veintitres': 23, 'veintitrés': 23, 'veinticuatro': 24, 'veinticinco': 25,
    'veintiseis': 26, 'veintiséis': 26, 'veintisiete': 27, 'veintiocho': 28, 'veintinueve': 29,
    'treinta': 30, 'cuarenta': 40, 'cincuenta': 50, 'sesenta': 60, 'setenta': 70,
    'ochenta': 80, 'noventa': 90, 'cien': 100
}

def parsear_numero_espanol(texto_num):
    """Convierte un string numérico o en letra a entero."""
    if not isinstance(texto_num, str):
        return None
        
    texto_num = texto_num.lower().strip()
    
    # Si ya es un dígito
    if texto_num.isdigit():
        return int(texto_num)
    
    # Si está en el diccionario directo
    if texto_num in str_num:
        return str_num[texto_num]
    
    # Manejo de compuestos como "treinta y cinco"
    parts = texto_num.split(' y ')
    if len(parts) == 2:
        decena = parts[0]
        unidad = parts[1]
        if decena in str_num and unidad in str_num:
            return str_num[decena] + str_num[unidad]
            
    return None

In [4]:
def extraer_victimas_nlp(texto):
    if not isinstance(texto, str):
        return None
    
    # 1. Limpieza básica para evitar leer encabezados o pies de página
    lines = texto.split('\n')
    cleaned_lines = []
    for line in lines:
        # Filtramos metadatos comunes en este dataset
        if "FOTO POR" in line or "Actualizado el" in line or "GRUPO ARMADO RELACIONADO" in line:
            continue
        cleaned_lines.append(line)
    
    # Unimos y tomamos los primeros 1200 caracteres (donde suele estar la descripción del hecho)
    # Convertimos a minúsculas para facilitar la búsqueda
    intro_text = "\n".join(cleaned_lines)[:1200].lower()
    
    # 2. Definición de componentes Regex
    # Captura números escritos o dígitos
    num_str = r'(?:\d+|uno|una|dos|tres|cuatro|cinco|seis|siete|ocho|nueve|diez|once|doce|trece|catorce|quince|dieciseis|diecisiete|dieciocho|diecinueve|veinte|veintiuno|veintidos|veintitres|veinticuatro|veinticinco|veintiseis|veintisiete|veintiocho|veintinueve|treinta|cuarenta|cincuenta|sesenta|setenta|ochenta|noventa|cien|\w+ y \w+)'
    # Captura sustantivos relevantes
    noun_str = r'(?:personas|víctimas|campesinos|indígenas|hombres|mujeres|pobladores|muertos|ciudadanos|trabajadores|mineros|pescadores|jóvenes)'
    
    # 3. Patrón Especial: Suma (Ej: "17 hombres y tres mujeres")
    sum_pat = fr'({num_str})\s+(?:{noun_str})\s+y\s+({num_str})\s+(?:{noun_str})'
    match_sum = re.search(sum_pat, intro_text)
    if match_sum:
        # Verificamos si hay palabras de violencia cerca para confirmar que son víctimas
        start, end = match_sum.span()
        contexto = intro_text[max(0, start-50):min(len(intro_text), end+50)]
        palabras_clave = ['identificados', 'asesinaron', 'mataron', 'muertos', 'cuerpos', 'víctimas', 'masacre']
        
        if any(x in contexto for x in palabras_clave):
            v1 = parsear_numero_espanol(match_sum.group(1))
            v2 = parsear_numero_espanol(match_sum.group(2))
            if v1 is not None and v2 is not None:
                return v1 + v2

    # 4. Patrones Estándar (Ordenados por prioridad)
    patterns = [
        # "asesinó a X personas"
        fr'(?:asesinó|asesinaron|mató|mataron|ultimaron|muerte|acribillaron|masacraron) a ({num_str}) {noun_str}',
        
        # "sacaron a X... y las asesinaron"
        fr'sacaron.*?a ({num_str}) {noun_str}.*?y (?:las|los) (?:asesinaron|mataron|ultimaron)',
        
        # "masacre de X personas"
        fr'masacre de ({num_str}) {noun_str}',
        
        # "dejando X muertos"
        fr'dejando (?:un saldo de )?({num_str}) {noun_str}',
        fr'dejando (?:un saldo de )?({num_str}) (?:muertos|asesinados|víctimas)',
        
        # "X personas fueron asesinadas"
        fr'({num_str}) {noun_str} (?:fueron|resultaron) (?:asesinadas|muertas|masacradas|ultimadas|acribilladas)',
        
        # "identificados X cuerpos"
        fr'identificados ({num_str}) (?:cuerpos|personas|víctimas)',
        
        # "asesinaron a X" (sin sustantivo, pero cuidando que no sea parte de una palabra)
        fr'(?:asesinó|asesinaron|mató|mataron) a ({num_str})(?![a-zA-Z])', 
        # 1. "reportes registraron 164 víctimas" (prioridad sobre el conteo inicial)
        fr'registr(?:aron|ó) ({num_str}) {noun_str}',
        
        # 2. "Murieron en total nueve personas"
        fr'murieron en total ({num_str}) {noun_str}',
        
        # 3. "Se llevaron a 14 personas para asesinarlas"
        fr'se llevaron a ({num_str}) {noun_str} para (?:asesinarlas|matarlas)',
        
        # 4. "asesinaron al menos a 15 personas" (cubre "al menos a")
        fr'(?:asesinó|asesinaron|mató|mataron) al menos a ({num_str}) {noun_str}',
        
        # 5. "otras cuatro víctimas"
        fr'otras ({num_str}) {noun_str}',
    ]
    
    for pat in patterns:
        match = re.search(pat, intro_text)
        if match:
            val = parsear_numero_espanol(match.group(1))
            if val:
                return val
            
    return None

In [5]:

def contar_nombres(valor):
    # Manejar valores nulos o vacíos
    if valor is None:
        return 0
    
    # Si es un array numpy o lista, convertirlo a string
    if isinstance(valor, (list, np.ndarray)):
        if len(valor) == 0:
            return 0
        valor_str = str(valor)
    elif pd.isna(valor):
        return 0
    else:
        valor_str = str(valor)
    
    # Casos sin información
    if valor_str in ["['-']", "['<p>-</p>']", "['<div>-</div>']", "['']", "[]"]:
        return 0
    if 'sin información' in valor_str.lower() or 'no hay información' in valor_str.lower():
        return 0
    if valor_str.strip() == '-':
        return 0
    
    nombres = []
    
    # Normalizar: reemplazar \\n con newline real
    valor_str = valor_str.replace('\\\\n\\\\n', '\n\n').replace('\\n\\n', '\n\n')
    
    # Patrón 1: Separados por newlines con tags <p> o <div>
    if '\n\n<p>' in valor_str or '\n\n<div>' in valor_str or '</p>\n\n' in valor_str or '</div>\n\n' in valor_str:
        matches = re.findall(r'<(?:p|div)>([^<]+)</(?:p|div)>', valor_str)
        for match in matches:
            nombre = match.strip()
            nombre = re.sub(r'&nbsp;', ' ', nombre)
            if nombre and nombre != '-' and len(nombre) > 1:
                nombres.append(nombre)
    
    # Patrón 2: Separados por comillas (formato array de strings)
    elif "' '" in valor_str or "'\n '" in valor_str or "'\n  '" in valor_str:
        partes = re.split(r"'\s*\n?\s*'", valor_str)
        for parte in partes:
            nombre = re.sub(r'<[^>]+>', '', parte)
            nombre = re.sub(r'&nbsp;', ' ', nombre)
            nombre = nombre.strip("'\" \n\t[]")
            if nombre and nombre != '-' and len(nombre) > 1:
                nombres.append(nombre)
    
    # Patrón 3: Un solo tag o formato simple
    else:
        matches = re.findall(r'<(?:p|div)>([^<]+)</(?:p|div)>', valor_str)
        if matches:
            for match in matches:
                nombre = match.strip()
                nombre = re.sub(r'&nbsp;', ' ', nombre)
                if nombre and nombre != '-' and len(nombre) > 1:
                    nombres.append(nombre)
        else:
            nombre = re.sub(r'<[^>]+>', '', valor_str)
            nombre = re.sub(r'&nbsp;', ' ', nombre)
            nombre = nombre.strip("'\"[] \n\t")
            if nombre and nombre != '-' and len(nombre) > 1:
                nombres.append(nombre)
    
    return len(nombres)


In [6]:
df = pd.read_parquet(raw/'rutas_del_conflicto'/'massacres.parquet')

df['victimas_detectadas_nlp'] = df['texto_noticia'].apply(extraer_victimas_nlp)

df['num_victimas_por_nombre'] = df['listado_victimas'].apply(contar_nombres)

df['victimas'] = df['victimas_detectadas_nlp'].fillna(df['num_victimas_por_nombre'])

df['victimas'] = np.where((df['victimas'] < 3) & (df['num_victimas_por_nombre'] >= 3), df['num_victimas_por_nombre'], df['victimas'])

In [7]:
for index, row in df.query("victimas < 3").iterrows():
    print(f"Index: {index}\nNoticia: \n{row['texto_noticia']}")
    print("\n\n")

Index: 31
Noticia: 
FOTO POR: ALCALDÍA DE LA UNIÓN.
Actualizado el: Lun, 10/14/2019 - 23:41
En la noche del 23 de agosto de 1988, un grupo de hombres armados detuvo una camioneta en zona rural de La Unión, Nariño, y dispararon contra los cuatro ocupantes. Las víctimas eran comerciantes que se dirigían desde Ipiales hacia el Cauca. Los cadáveres fueron arrojados al río Mayo, cerca del lugar. 
  No hay certeza sobre qué grupo armado perpetró esta masacre. Desde finales de los años sesenta en la zona ha delinquido el Comando Conjunto Occidental de las Farc, que desde 2011 adoptó el nombre de Bloque Alfonso Cano. En los años ochenta también tuvo influencia el Eln.
GRUPO ARMADO RELACIONADO:
GRUPO ARMADO NO IDENTIFICADO
NULL



Index: 34
Noticia: 
FOTO POR: ALCALDÍA LOS CÓRDOBAS
Actualizado el: Mié, 10/02/2019 - 22:48
El 12 de noviembre de 1988 un grupo de paramilitares sin identificar ingresaron a finca La Puya, en el municipio de Los Córdobas en el departamento de Córdoba. Allí decapitaron

In [8]:
df.loc[31, "victimas"] = 4
df.loc[34, "victimas"] = 6
df.loc[82, "victimas"] = 35
df.loc[89, "victimas"] = 8
df.loc[96, "victimas"] = 4
df.loc[185, "victimas"] = 12
df.loc[220, "victimas"] = 6
df.loc[236, "victimas"] = 9
df.loc[311, "victimas"] = 6
df.loc[319, "victimas"] = 11
df.loc[321, "victimas"] = 12
df.loc[334, "victimas"] = 4
df.loc[419, "victimas"] = 6
df.loc[425, "victimas"] = 19
df.loc[457, "victimas"] = 17
df.loc[502, "victimas"] = 5
df.loc[517, "victimas"] = 4
df.loc[529, "victimas"] = 4
df.loc[585, "victimas"] = 60
df.loc[619, "victimas"] = 7
df.loc[620, "victimas"] = 8
df.loc[654, "victimas"] = 15
df.loc[704, "victimas"] = 6
df.loc[705, "victimas"] = 7
df.loc[710, "victimas"] = 5
df.loc[102, "victimas"] = 5
df.loc[257, "victimas"] = 4
df.loc[385, "victimas"] = 4
df.loc[429, "victimas"] = 31


In [9]:
# Top 20 masacres con más víctimas
for index, row in df.nlargest(20, 'victimas').iterrows():
    print(f"Index: {index}\nNoticia: \n{row['texto_noticia']}\n Victimas reportadas: {row['victimas']}")
    print("\n\n")

Index: 9
Noticia: 
FOTO POR: REVISTA SEMANA
Actualizado el: Mié, 10/02/2019 - 22:48
En los primeros días de diciembre de 1985 comenzaron los rumores de que en el corregimiento Tacueyó, del municipio de Toribío, se estaba cometiendo una matanza sin precedentes. Entre noviembre de 1985 y enero de 1986, Fedor Rey, alias 'Javier Delgado' y Hernando Pizarro Leóngomez comandantes de un grupo guerrillero disidente de las Farc, asesinaron a más de cien de sus compañeros.
Según las investigaciones judiciales, fueron asesinadas 125 personas, pero reportes de prensa de la época registraron 164 víctimas. La mayoría eran jóvenes campesinos que habían ingresado recientemente a las filas de la columna Ricardo Franco. Muchos también eran universitarios que fueron llamados por ‘Delgado’ hasta sus campamentos con el único propósito de ser asesinados.
'El monstruo de los Andes', como se le conoció a ‘Delgado’, torturó a todas sus víctimas con métodos escalofriantes. Se encontraron cuerpos a los que les h

In [10]:
df.loc[536, "victimas"] = 98

In [11]:
# Diccionarios de configuración

CORRECCIONES_DEPARTAMENTO = {
    'Caqueta': 'Caquetá',
    'Tolíma': 'Tolima',
    'valle': 'Valle del Cauca'
}

MESES_ES = {
    'enero': '01', 'febrero': '02', 'marzo': '03', 'abril': '04',
    'mayo': '05', 'junio': '06', 'julio': '07', 'agosto': '08',
    'septiembre': '09', 'octubre': '10', 'noviembre': '11', 'diciembre': '12'
}

def limpiar_municipio_departamento(valor):
    """
    Separa municipio y departamento.
    Input:  'Municipio y departamento: Amalfi , Antioquia'
    Output: ('Amalfi', 'Antioquia')
    """
    texto = str(valor).replace('Municipio y departamento:', '').strip()
    partes = texto.rsplit(',', 1)
    
    if len(partes) == 2:
        municipio = partes[0].strip().rstrip('.').rstrip(',').strip()
        departamento = partes[1].strip()
    else:
        municipio = texto.strip()
        departamento = ''
    
    # Limpiar espacios extras
    municipio = re.sub(r'\s+,', ',', municipio)
    municipio = re.sub(r',\s+', ', ', municipio)
    
    return municipio, departamento

In [12]:
def limpiar_grupo_armado(valor):
    """
    Limpia grupo armado quitando prefijo y años.
    Input:  'Grupo Armado: Paramilitares , Paramilitares de Fidel Castaño (1982 - 1994)'
    Output: 'Paramilitares de Fidel Castaño'
    """
    texto = str(valor).replace('Grupo Armado:', '').strip()
    
    # Quitar años entre paréntesis
    texto = re.sub(r'\s*\(\d{4}\s*-\s*\d{0,4}\s*\)', '', texto)
    
    # Separar por coma y tomar el nombre del grupo
    partes = texto.split(',', 1)
    grupo = partes[1].strip() if len(partes) == 2 else partes[0].strip()
    
    return grupo if grupo else 'No identificado'

In [13]:
def limpiar_fecha(valor):
    """
    Limpia fecha quitando prefijo y manejando formato español.
    Input:  'Fecha: 1982-08-04' o 'Fecha: 22 de marzo de 1998'
    Output: '1982-08-04' o '1998-03-22'
    """
    texto = str(valor).replace('Fecha:', '').strip()
    
    # Si ya está en formato ISO
    if re.match(r'^\d{4}-\d{2}-\d{2}$', texto):
        return texto
    
    # Si está en formato español
    match = re.match(r'(\d{1,2})\s+de\s+(\w+)\s+de\s+(\d{4})', texto)
    if match:
        dia = match.group(1).zfill(2)
        mes = MESES_ES.get(match.group(2).lower(), '01')
        año = match.group(3)
        return f"{año}-{mes}-{dia}"
    
    return texto

## 3. Aplicar transformaciones

### 3.1 Separar municipio y departamento

In [14]:
# Aplicar función y crear dos columnas nuevas
df[['municipio_limpio', 'departamento']] = df['municipio'].apply(
    lambda x: pd.Series(limpiar_municipio_departamento(x))
)

# Corregir departamentos mal escritos
df['departamento'] = df['departamento'].replace(CORRECCIONES_DEPARTAMENTO)

print(f"Departamentos únicos: {df['departamento'].nunique()}")
df[['municipio', 'municipio_limpio', 'departamento']].head(10)

Departamentos únicos: 28


,municipio,municipio_limpio,departamento
0,"Municipio y departamento: Amalfi , Antioquia",Amalfi,Antioquia
1,"Municipio y departamento: Puerto Triunfo , Ant...",Puerto Triunfo,Antioquia
2,"Municipio y departamento: Remedios , Antioquia",Remedios,Antioquia
3,"Municipio y departamento: Almaguer , Cauca",Almaguer,Cauca
4,"Municipio y departamento: Inzá , Cauca",Inzá,Cauca
5,"Municipio y departamento: Jambaló , Cauca",Jambaló,Cauca
6,"Municipio y departamento: Páez , Cauca",Páez,Cauca
7,"Municipio y departamento: Turbo , Antioquia",Turbo,Antioquia
8,"Municipio y departamento: Silvia , Cauca",Silvia,Cauca
9,"Municipio y departamento: Toribío , Cauca",Toribío,Cauca


### 3.2 Limpiar grupo armado

In [15]:
df['grupo_armado_limpio'] = df['grupo_armado'].apply(limpiar_grupo_armado)

print(f"Grupos únicos: {df['grupo_armado_limpio'].nunique()}")
df[['grupo_armado', 'grupo_armado_limpio']].head(10)

Grupos únicos: 42


,grupo_armado,grupo_armado_limpio
0,"Grupo Armado: Paramilitares , Paramilitares de...",Paramilitares de Fidel Castaño
1,"Grupo Armado: Paramilitares , Autodefensas de ...",Autodefensas de Puerto Boyacá
2,"Grupo Armado: Paramilitares , Paramilitares de...",Paramilitares de Fidel Castaño
3,"Grupo Armado: Grupo armado no identificado , G...",Grupo armado no identificado
4,"Grupo Armado: Grupo armado no identificado , G...",Grupo armado no identificado
5,"Grupo Armado: Guerrilla , Fuerzas Armadas Revo...",Fuerzas Armadas Revolucionarias de Colombia - ...
6,"Grupo Armado: Guerrilla , Fuerzas Armadas Revo...",Fuerzas Armadas Revolucionarias de Colombia - ...
7,"Grupo Armado: Guerrilla , Grupo guerrillero no...",Grupo guerrillero no identificado
8,"Grupo Armado: Grupo armado no identificado , G...",Grupo armado no identificado
9,"Grupo Armado: ,",No identificado


### 3.3 Convertir fecha a datetime

In [16]:
# Convertir a datetime
df['fecha'] = pd.to_datetime(df['fecha'].apply(limpiar_fecha), format='%Y-%m-%d')

# Agregar a frecuencia mensual (primer día del mes)
df['fecha'] = df['fecha'].dt.to_period('M').dt.to_timestamp().dt.date

## 4. Crear DataFrame limpio

In [ ]:
df_limpio = df[['municipio_limpio', 'departamento', 'grupo_armado_limpio', 'fecha', 'victimas']].copy()
df_limpio.columns = ['municipio', 'departamento', 'grupo_armado', 'fecha', 'victimas']

df_limpio.to_parquet(
        temp / 'rdc' / 'raw_massacres.parquet',
        index=False
    )

In [21]:
df_limpio[['municipio','departamento']].drop_duplicates().to_excel(
    temp / 'rdc' / 'municipios_departamentos_massacres.xlsx',)

In [28]:
df_limpio

,municipio,departamento,grupo_armado,fecha,victimas
0,Amalfi,Antioquia,Paramilitares de Fidel Castaño,1982-08-01,9.0
1,Puerto Triunfo,Antioquia,Autodefensas de Puerto Boyacá,1982-09-01,5.0
2,Remedios,Antioquia,Paramilitares de Fidel Castaño,1983-08-01,20.0
3,Almaguer,Cauca,Grupo armado no identificado,1984-06-01,4.0
4,Inzá,Cauca,Grupo armado no identificado,1985-05-01,4.0
...,...,...,...,...,...
737,Cali,Valle del Cauca,"Bandas Criminales Emergentes, Bacrim",2013-11-01,9.0
738,Cali,Valle del Cauca,"Bandas Criminales Emergentes, Bacrim",2014-10-01,8.0
739,Amalfi,Antioquia,"Bandas Criminales Emergentes, Bacrim",2014-12-01,7.0
740,Magüí Payán,Nariño,"Ejército de Liberación Nacional, Eln",2017-11-01,13.0


In [37]:
df_mun_comp = pd.read_excel(raw/'rutas_del_conflicto'/'municipios_departamentos_con_codigo.xlsx').dropna().reset_index(drop=True)
df_mun_dup = pd.read_excel(raw/'rutas_del_conflicto'/'municipios_sin_codigo_expandidos.xlsx')[['municipio_original','cantidad_municipios','codigo_municipio']]

In [54]:
df_incompleto_duplicados = df_limpio.merge(df_mun_dup, left_on='municipio', right_on='municipio_original', how='left').dropna()[['fecha','victimas','cantidad_municipios','codigo_municipio']]
df_incompleto_duplicados['victimas_total_ajustado'] = round(df_incompleto_duplicados['victimas'] / df_incompleto_duplicados['cantidad_municipios'],2)
df_incompleto_duplicados.drop(columns=['cantidad_municipios','victimas'], inplace=True)
df_incompleto_duplicados.rename(columns={'victimas_total_ajustado':'victimas'}, inplace=True)

df_incompleto_completo = df_limpio.merge(df_mun_comp, on=['municipio','departamento'], how='left').dropna().reset_index(drop=True)[['fecha','codigo_municipio','victimas']]

In [55]:
df_completo = pd.concat([df_incompleto_completo, df_incompleto_duplicados], ignore_index=True).reset_index(drop=True)
df_completo 


,fecha,codigo_municipio,victimas
0,1982-08-01,5031.0,9.0
1,1982-09-01,5591.0,5.0
2,1983-08-01,5604.0,20.0
3,1984-06-01,19022.0,4.0
4,1985-05-01,19355.0,4.0
...,...,...,...
757,2003-02-01,85015.0,30.0
758,2003-06-01,17614.0,2.0
759,2003-06-01,17777.0,2.0
760,2004-03-01,81220.0,8.0


In [57]:
df_completo.groupby(['fecha','codigo_municipio'])['victimas'].sum().reset_index().to_parquet(
        temp / 'rdc' / 'massacres.parquet',
        index=False
    )